In [37]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import tensorflow_datasets as tfds

from wordcloud import WordCloud
import matplotlib as plt
import seaborn as sns

import re
from sklearn.decomposition import PCA
from wordcloud import WordCloud
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

#silence TF
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [38]:
df = pd.read_csv('games-features-cleaned.csv')
df.head()

,Unnamed: 0,QueryName,Metacritic,GenreIsNonGame,GenreIsIndie,GenreIsAction,GenreIsAdventure,GenreIsCasual,GenreIsStrategy,GenreIsRPG,...,Finnish,Ukrainian,Romanian,Thai,Greek,Bulgarian,Arabic,Slovakian,num_lang_audio_support,num_languages
0,0,Counter-Strike,88,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0.0,8
1,2,Day of Defeat,79,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0.0,5
2,6,Half-Life,96,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0.0,8
3,7,Counter-Strike: Condition Zero,65,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0.0,8
4,9,Half-Life: Blue Shift,71,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0.0,3


In [39]:
df['Reviews'].unique()

array([' ',
       '... RO is also one of the markets most unique new shooters4/5 Stars - GamespyTripwires attention to accessible realism is unrivaled- Computer Gaming WorldRed Orchestra offers exactly the right mixture of action and realism- PC PowerPlayRed Orchestra scores Best Multiplayer Game for 2006 over at VooDoo Extreme.Read the full review here.',
       'IGN Awards Arx Fatalis an 8/10The spell casting system (...) is probably the most fun Ive had with magic in a game in a long time.RPGDot gives Arx Fatalis a 91%Clever dungeon design exciting story and challenging puzzles makes this the most intelligent RPG around',
       'It may not replace Star Craft in terms of stature - but its damn close. The multiplayer matches are suitably intense and the single-player campaign delivers everything in terms of both expectation and intention.90 - Gamers HellAn extremely lengthy deep and beautiful strategy experience. From the very start youll find yourself drawn in and playing for quite

RegEx Preprocessor, Encoding and Encoding Map Functions from Cornelia's Week 10 Notebook

In [40]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [41]:
# define function for token encoder
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encode(text)
    return encoded_text, label

#  wrap the encode function to a TF Operator
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
                          Tout=(tf.int64, tf.int64))

Data Preprocessing

In [42]:
df['DetailedDescrip'] = df['DetailedDescrip'].apply(preprocessor)
df['Reviews'] = df['Reviews'].apply(preprocessor)
df['Reviews'].head()

0     
1     
2     
3     
4     
Name: Reviews, dtype: object